In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/NLP-DL

/content/drive/MyDrive/NLP-DL


In [4]:
!ls

LSTM  RNN


### RNN

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import pickle

In [7]:
char_mappings = pickle.load(open("RNN/char_mappings.pkl", "rb"))

In [8]:
idx_to_char = char_mappings['idx_to_char']
char_to_idx = char_mappings['char_to_idx']

In [9]:
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.hidden_dim = hidden_dim

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        device = next(self.parameters()).device  # Get the device of the model
        return torch.zeros(1, batch_size, self.hidden_dim, device=device)

In [10]:
rnn_embedding_dim = 128
rnn_hidden_dim = 256
rnn_model = RNNModel(43, rnn_embedding_dim, rnn_hidden_dim)

In [11]:
rnn_model = rnn_model.to(device)

In [12]:
rnn_model.load_state_dict(torch.load('RNN/rnn.pt'))

<All keys matched successfully>

In [13]:
def generate_lyrics(model, start_letter, max_length=100, temperature=1.0):
    model.eval()

    # Start with the given letter
    input_seq = torch.tensor([char_to_idx[start_letter]], dtype=torch.long).unsqueeze(0).to(device)
    hidden = model.init_hidden(1)
    generated = [start_letter]

    with torch.no_grad():
        for _ in range(max_length):
            output, hidden = model(input_seq, hidden)

            # Apply temperature scaling before softmax
            probs = torch.softmax(output[-1] / temperature, dim=-1)

            # Sample the next character
            next_char_idx = torch.multinomial(probs, 1).item()
            next_char = idx_to_char[next_char_idx]
            generated.append(next_char)

            # Prepare next input
            input_seq = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

            if next_char == '\n':  # Stop at newline (optional)
                break

    return ''.join(generated)


In [24]:
print(generate_lyrics(rnn_model, start_letter='a', max_length=50, temperature=0.8))
print(generate_lyrics(rnn_model, start_letter='t', max_length=50, temperature=1.5))

and you offi could ithere viemy don't got down all 
thigghing,but i berighberwixasbusry, cries woldwont


### LSTM

In [15]:
import pickle

In [16]:
vocab = pickle.load(open("LSTM/lstm-word-vocab.pkl", "rb"))

In [17]:
import torch.nn as nn
import torch

In [18]:
class LyricsLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(LyricsLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

In [19]:
embedding_dim = 64
hidden_dim = 128
num_layers = 2
vocab_size = len(vocab)

# Initialize model
model = LyricsLSTM(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)

In [20]:
model.load_state_dict(torch.load('LSTM/lstm.pt'))

<All keys matched successfully>

In [21]:
word_mappings = pickle.load(open("LSTM/lstm-idx-mapping.pkl", "rb"))
idx_to_word = word_mappings['idx_to_word']
word_to_idx = word_mappings['word_to_idx']

#### Inference

In [22]:
import torch
import random

def generate_lyrics_using_lstm(starting_letter, max_words=50, temperature=1.0):
    filtered_vocab = [word for word in vocab if word.startswith(starting_letter)]
    if not filtered_vocab:
        return "No words found starting with this letter."

    words = [random.choice(filtered_vocab)]

    hidden = (torch.zeros(num_layers, 1, hidden_dim).to(device),
              torch.zeros(num_layers, 1, hidden_dim).to(device))

    for _ in range(max_words - 1):
        input_seq = torch.tensor([[word_to_idx[words[-1]]]], dtype=torch.long).to(device)
        output, hidden = model(input_seq, hidden)

        output = output / temperature
        probabilities = torch.nn.functional.softmax(output, dim=1)

        predicted_idx = torch.multinomial(probabilities, 1).item()

        next_word = idx_to_word.get(predicted_idx, "<UNK>")
        words.append(next_word)

    return ' '.join(words)

In [23]:
print(generate_lyrics_using_lstm('a', max_words=10, temperature=0.8))

anakin my my clots, she was our cry on his
